# DB-Chat

Ask DB is a chat based interface to talk with a database in natural language. It can be used to make queries on a database that can help users get information and valuable insights.

## Gemini LLM

In [1]:
# Imports
import os
from dotenv import dotenv_values 
from langchain_google_genai import ChatGoogleGenerativeAI

/home/naveen/micromamba/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Read environment file
GOOGLE_API_KEY = dotenv_values(os.path.expanduser('~/.google'))['GOOGLE_AI_API_KEY']

In [137]:
# LLM Object
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash', google_api_key=GOOGLE_API_KEY, temperature=0.2)

In [138]:
# Test LLM
llm.invoke("Write a haiku about nature")

AIMessage(content="Green leaves whisper soft,\nSunlight dances on the stream,\nPeace in nature's heart. \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0cdd93e1-79c5-4bc2-87bf-2ff84584bce3-0')

## Database Agent

In [139]:
# Imports
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.agent_toolkits import create_sql_agent

In [140]:
# Create db agent toolkit
db_user = 'root'
db_password = 'admin'
db_host = 'localhost'
db_name = 'askdb'
db = SQLDatabase.from_uri(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [312]:
# DB Agent
agent_executor = create_sql_agent(llm, toolkit=toolkit, verbose=True, agent_type='tool-calling')

In [489]:
# Test db agent
agent_executor.invoke("List all the brand names")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


brand, category, discount, product, product_name
Invoking: `sql_db_schema` with `{'table_names': 'brand'}`



CREATE TABLE brand (
	brand_id INTEGER(11) NOT NULL, 
	name VARCHAR(128) NOT NULL, 
	PRIMARY KEY (brand_id)
)COLLATE utf8mb4_uca1400_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from brand table:
brand_id	name
1	Nike
2	Adidas
3	Levi
*/
Invoking: `sql_db_query` with `{'query': 'SELECT name FROM brand'}`


[('Nike',), ('Adidas',), ('Levi',), ('Gucci',)]The brand names are Nike, Adidas, Levi, and Gucci.

> Finished chain.


{'input': 'List all the brand names',
 'output': 'The brand names are Nike, Adidas, Levi, and Gucci.'}

## Few Shot Learning

The LLM does not understand the database schema so it makes mistakes, like not multiplying price by quantity when asking for total price of some products in my sample database.
I have prepared some data based on the mistakes it makes for few shot learning to help the model learn context.

#### Load Data

In [507]:
# Import module
import json

In [515]:
few_shot = []
with open('./vector-db/few-shot-data.json' , 'r') as f:
    few_shot = json.load(f)

In [517]:
blobs = [ " ".join(sample.values()) for sample in few_shots]

#### Generate Embeddings

In [519]:
# Import huggingface embedding lib
from langchain_huggingface import HuggingFaceEmbeddings

In [520]:
# Embedder
embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/home/naveen/micromamba/envs/ml/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### Add Embeddings to Vector Store

In [521]:
# Import vector store
from langchain.vectorstores import Chroma

In [523]:
# Generate vector store
vectorstore = Chroma.from_texts(blobs, embedding=embedding, metadatas=few_shots)

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 2


#### Similarity Matching

In [524]:
# Import matcher
from langchain.prompts import SemanticSimilarityExampleSelector

In [535]:
# Selector object
example_selector = SemanticSimilarityExampleSelector(vectorstore=vectorstore, k=5, input_keys=["input"])

In [536]:
example_selector.select_examples({"input": "What color are available for all gucci items"})

[{'answer': 'The available colors for Adidas Boots are: White.',
  'input': 'What colors are available for Gucci boots?',
  'query': "SELECT DISTINCT color FROM product WHERE product_name_id = (SELECT product_name_id FROM product_name WHERE name = 'Boots') AND brand_id = (SELECT brand_id FROM brand WHERE name = 'Gucci')"},
 {'answer': 'The available brands are Nike, Adidas, Levi, and Gucci.',
  'input': 'What brands are available?',
  'query': 'select name from brand'},
 {'answer': 'The available colors for Shorts of brand Nike are Red.',
  'input': 'What colors are available for Shorts of brand Nike?',
  'query': "SELECT DISTINCT color FROM product WHERE brand_id = (SELECT brand_id FROM brand WHERE name = 'Nike') AND product_name_id = (SELECT product_name_id FROM product_name WHERE name = 'Shorts')"},
 {'answer': 'There are 3 red Nike shirts available.',
  'input': 'How many red Nike Shirts are available.',
  'query': "SELECT sum(quantity) FROM product WHERE color = 'Red' AND brand_id

#### Custom Prompt

In [537]:
print("System: You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct mysql query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\n\nIf the question does not seem related to the database, just return \"I don't know\" as the answer.\n\nHuman: List all the brand names\nAI: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.")

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct mysql query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I 

In [538]:
# Import prompt templates
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder

In [539]:
# System prefix
system_prefix = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST always double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
Only have single quotes on any sql command sent to the engine.
Do not escape single quotes on any sql command sent to the engine.
Do not return raw query as result after generating it, execute it and return result.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Whenever you return any price, use rupees (Rs.) as currency and use two comma separator format for formatting the price.

Here are some examples of user inputs and their corresponding SQL queries:"""

In [540]:
# Example prompt template
example_prompt = PromptTemplate(
    input_variables=['input', 'query', 'answer'],
    template='Human: {input}\nSQL Query: {query}\nAI:{answer}'
)

In [541]:
# Create prompt template for few shots
few_shot_prompt = FewShotPromptTemplate (
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=system_prefix,
    suffix="\n",
    input_variables=["input",  "top_k", "dialect",],
)

In [542]:
# Full prompt
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        ("ai", "I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables."),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [543]:
# Example formatted prompt
sample_prompt = full_prompt.invoke(
    {
        "input": "What is the total price of all nike products",
        "top_k": 5,
        "dialect": "MariaDB",
        "agent_scratchpad": [],
    }
)
print(sample_prompt.to_string())

System: 
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct MariaDB query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST always double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
Only have single quotes on any sql command sent to the engine.
Do not escape single quotes on any sql command sent to the engine.
Do not return raw

In [544]:
# New agent after few shots learning
agent = create_sql_agent(llm, toolkit=toolkit, prompt=full_prompt, verbose=True, agent_type='tool-calling')

#### Test Agent

In [545]:
# Test new db agent
agent.invoke("What colors are available for Gucci boots?")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


brand, category, discount, product, product_name
Invoking: `sql_db_schema` with `{'table_names': 'brand, product, product_name'}`



CREATE TABLE brand (
	brand_id INTEGER(11) NOT NULL, 
	name VARCHAR(128) NOT NULL, 
	PRIMARY KEY (brand_id)
)COLLATE utf8mb4_uca1400_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from brand table:
brand_id	name
1	Nike
2	Adidas
3	Levi
*/


CREATE TABLE product (
	product_id INTEGER(11) NOT NULL, 
	quantity INTEGER(11) NOT NULL, 
	price INTEGER(11) NOT NULL, 
	color VARCHAR(64), 
	product_name_id INTEGER(11) NOT NULL, 
	category_id INTEGER(11) NOT NULL, 
	brand_id INTEGER(11) NOT NULL, 
	PRIMARY KEY (product_id), 
	CONSTRAINT product_ibfk_1 FOREIGN KEY(category_id) REFERENCES category (category_id), 
	CONSTRAINT product_ibfk_2 FOREIGN KEY(brand_id) REFERENCES brand (brand_id), 
	CONSTRAINT product_ibfk_3 FOREIGN KEY(product_name_id) REFERENCES p

{'input': 'What colors are available for Gucci boots?',
 'output': 'The available colors for Gucci Boots are: White. \n'}

In [546]:
agent.invoke("How much is the price of the Nike inventory")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


brand, category, discount, product, product_name
Invoking: `sql_db_schema` with `{'table_names': 'brand, product'}`



CREATE TABLE brand (
	brand_id INTEGER(11) NOT NULL, 
	name VARCHAR(128) NOT NULL, 
	PRIMARY KEY (brand_id)
)COLLATE utf8mb4_uca1400_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from brand table:
brand_id	name
1	Nike
2	Adidas
3	Levi
*/


CREATE TABLE product (
	product_id INTEGER(11) NOT NULL, 
	quantity INTEGER(11) NOT NULL, 
	price INTEGER(11) NOT NULL, 
	color VARCHAR(64), 
	product_name_id INTEGER(11) NOT NULL, 
	category_id INTEGER(11) NOT NULL, 
	brand_id INTEGER(11) NOT NULL, 
	PRIMARY KEY (product_id), 
	CONSTRAINT product_ibfk_1 FOREIGN KEY(category_id) REFERENCES category (category_id), 
	CONSTRAINT product_ibfk_2 FOREIGN KEY(brand_id) REFERENCES brand (brand_id), 
	CONSTRAINT product_ibfk_3 FOREIGN KEY(product_name_id) REFERENCES product_name (p

{'input': 'How much is the price of the Nike inventory',
 'output': 'The price of the Nike inventory is Rs. 56,135. \n'}

In [547]:
agent.invoke("If I sell all my Gucci shirts that are on discount, how much revenue will my store generate?")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


brand, category, discount, product, product_name
Invoking: `sql_db_schema` with `{'table_names': 'brand, category, discount, product, product_name'}`



CREATE TABLE brand (
	brand_id INTEGER(11) NOT NULL, 
	name VARCHAR(128) NOT NULL, 
	PRIMARY KEY (brand_id)
)COLLATE utf8mb4_uca1400_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from brand table:
brand_id	name
1	Nike
2	Adidas
3	Levi
*/


CREATE TABLE category (
	category_id INTEGER(11) NOT NULL, 
	name VARCHAR(128) NOT NULL, 
	PRIMARY KEY (category_id)
)COLLATE utf8mb4_uca1400_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from category table:
category_id	name
1	Topwear
2	Bottomwear
3	Footwear
*/


CREATE TABLE discount (
	discount_id INTEGER(11) NOT NULL, 
	percent INTEGER(11) NOT NULL, 
	product_id INTEGER(11) NOT NULL, 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discount_ibfk_1 FOREIGN KEY(product_id) REFERENCES p

{'input': 'If I sell all my Gucci shirts that are on discount, how much revenue will my store generate?',
 'output': 'Your store will generate Rs. 2,047.50 in revenue if you sell all your Gucci shirts that are on discount. \n'}

In [548]:
agent.invoke("If I sell all my Adidas items that are on discount, how much revenue will my store generate?")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


brand, category, discount, product, product_name
Invoking: `sql_db_schema` with `{'table_names': 'brand, discount, product, product_name'}`



CREATE TABLE brand (
	brand_id INTEGER(11) NOT NULL, 
	name VARCHAR(128) NOT NULL, 
	PRIMARY KEY (brand_id)
)COLLATE utf8mb4_uca1400_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from brand table:
brand_id	name
1	Nike
2	Adidas
3	Levi
*/


CREATE TABLE discount (
	discount_id INTEGER(11) NOT NULL, 
	percent INTEGER(11) NOT NULL, 
	product_id INTEGER(11) NOT NULL, 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discount_ibfk_1 FOREIGN KEY(product_id) REFERENCES product (product_id)
)COLLATE utf8mb4_uca1400_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discount table:
discount_id	percent	product_id
1	10	1
2	15	2
3	20	3
*/


CREATE TABLE product (
	product_id INTEGER(11) NOT NULL, 
	quantity INTEGER(11) NOT NULL, 
	price INTEGER(

{'input': 'If I sell all my Adidas items that are on discount, how much revenue will my store generate?',
 'output': 'Your store will generate Rs. 14,776.00 in revenue if you sell all your Adidas items that are on discount. \n'}

In [549]:
agent.invoke("If I sell all my boots that are on discount, how much revenue will my store generate?")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


brand, category, discount, product, product_name
Invoking: `sql_db_schema` with `{'table_names': 'brand, category, discount, product, product_name'}`



CREATE TABLE brand (
	brand_id INTEGER(11) NOT NULL, 
	name VARCHAR(128) NOT NULL, 
	PRIMARY KEY (brand_id)
)COLLATE utf8mb4_uca1400_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from brand table:
brand_id	name
1	Nike
2	Adidas
3	Levi
*/


CREATE TABLE category (
	category_id INTEGER(11) NOT NULL, 
	name VARCHAR(128) NOT NULL, 
	PRIMARY KEY (category_id)
)COLLATE utf8mb4_uca1400_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from category table:
category_id	name
1	Topwear
2	Bottomwear
3	Footwear
*/


CREATE TABLE discount (
	discount_id INTEGER(11) NOT NULL, 
	percent INTEGER(11) NOT NULL, 
	product_id INTEGER(11) NOT NULL, 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discount_ibfk_1 FOREIGN KEY(product_id) REFERENCES p

{'input': 'If I sell all my boots that are on discount, how much revenue will my store generate?',
 'output': 'Your store will generate Rs. 0.00 in revenue if you sell all your boots that are on discount. \n'}